In [116]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score,KFold
from xgboost import XGBRegressor
import seaborn
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
df_full = pd.concat([df, test]).reset_index(drop=True)
noncategorial = list(
df_full.select_dtypes(include=['float64', 'int64']).columns)
categorial = list(df_full.select_dtypes(include=['object']).columns)

In [117]:
df_full.loc[(df_full["BsmtExposure"].isna()) & (df_full["BsmtCond"].isna())  & (df_full.BsmtQual.isna())
           & (df_full.BsmtFinType1.isna())  & (df_full.BsmtFinType2.isna()) & (df_full.TotalBsmtSF == 0), 
           ['BsmtExposure','BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2']] = 'NoBsmt'
df_full.BsmtCond.fillna('TA',inplace=True)
df_full.BsmtExposure.fillna(df_full[(df_full.BsmtCond == 'TA') 
                                    & (df_full.BsmtQual == 'Gd')].groupby(['BsmtCond','BsmtQual','BsmtExposure']).size().idxmax(axis=1)[2],inplace=True)
df_full.BsmtFinType2.fillna(df_full[(df_full.GarageCond == 'TA') & ( df_full.BsmtQual == 'Gd') 
        & ( df_full.BsmtFinType1 == 'GLQ')].groupby(['GarageCond','BsmtQual','BsmtFinType1','BsmtFinType2']).size().idxmax()[3],inplace=True)
df_full.BsmtFullBath.fillna(0,inplace=True)
df_full.BsmtHalfBath.fillna(0,inplace=True)
df_full.BsmtQual.fillna(df_full[((df_full.BsmtCond == 'TA') | (df_full.BsmtCond == 'Gd' )) &(df_full.BsmtFinType1 == 'Unf')  
        & ( df_full.BsmtFinType2 == 'Unf')].groupby(['BsmtCond','BsmtFinType1','BsmtFinType2','BsmtQual']).size().idxmax()[3]
                       ,inplace=True)
df_full["SalePrice"] = np.log1p(df_full['SalePrice'])

In [118]:

df_full.loc[(df_full["GarageCond"].isna()) & (df_full["GarageType"].isna()), ['GarageFinish','GarageType','GarageCond','GarageQual']] = 'NoGarage'
df_full.GarageCond.fillna(df_full[df_full.GarageType == 'Detchd']
                                               .groupby(['GarageType', 'GarageCond'])
                                               .size()
                                               .idxmax(axis=1)[1],inplace=True)
df_full.GarageFinish.fillna(df_full[df_full.GarageType == 'Detchd'].groupby(['GarageType','GarageFinish']).size()
                            .idxmax(axis=1)[1],inplace=True)
df_full.GarageQual.fillna(df_full[df_full.GarageType == 'Detchd']
                                   .groupby(['GarageType','GarageQual'])
                                    .size()
                                    .idxmax(axis=1)[1],inplace=True)
df_full.GarageYrBlt.fillna(df_full[df_full.GarageType == 'Detchd']
                                     .groupby(['GarageType','GarageYrBlt'])
                                     .size()
                                     .idxmax(axis=1)[1],inplace=True)
df_full.GarageCars.fillna(round(df_full[(df_full.GarageType == 'Detchd') & (df_full.GarageCond == 'TA')].groupby(['GarageType','GarageCond'])['GarageCars'].mean()[0],2),inplace=True)
df_full.GarageArea.fillna(df_full[(df_full.GarageType == 'Detchd') & ( df_full.GarageCond == 'TA')]
                                                .groupby(['GarageType','GarageCond','GarageArea'])
                                                .size()
                                                .idxmax(axis=1)[2],inplace=True)

In [119]:
df_full.Electrical.fillna(df_full.groupby('Electrical').size().idxmax(),inplace=True)
df_full.Exterior1st.fillna(df_full.groupby('Exterior1st').size().idxmax(),inplace=True)
df_full.Exterior2nd.fillna(df_full.groupby('Exterior2nd').size().idxmax(),inplace=True)
df_full.Functional.fillna(df_full.groupby('Functional').size().idxmax(),inplace=True)
df_full.KitchenQual.fillna(df_full.groupby('KitchenQual').size().idxmax(),inplace=True)
df_full.MSZoning.fillna('RL',inplace=True)
df_full.SaleType.fillna('WD',inplace = True)
df_full.Utilities.fillna('AllPub',inplace=True)

In [120]:
df_full.drop(['Street'], axis=1, inplace=True)
df_full.drop(['Fence'], axis=1, inplace=True)
df_full.groupby('Alley').size()
df_full.Alley.fillna('Grvl',inplace=True)
#df_full.Fence.fillna('MnPrv',inplace=True)
df_full.groupby('FireplaceQu').size()
df_full.FireplaceQu.fillna('Gd',inplace=True)
df_full.MiscFeature.fillna(df_full.groupby('MiscFeature').size().idxmax(),inplace=True)
df_full.PoolQC.fillna(df_full.groupby('PoolQC').size().idxmax(),inplace=True)
df_full.loc[(df_full["MasVnrArea"].isna() & (df_full["MasVnrType"].isna()), 'MasVnrType')] = 'None'
df_full.MasVnrArea.fillna(0,inplace=True)
df_full.MasVnrType.fillna(df_full['MasVnrType'][(df_full.MasVnrArea >= 100) & (df_full.MasVnrArea <= 150)].value_counts().index[0],inplace=True)
#df_full["LotFrontage"] = df_full.groupby("Neighborhood")["LotFrontage"].transform(
#    lambda x: x.fillna(x.median()))
df_full['MSSubClass'] = df_full['MSSubClass'].apply(str)
#Year and month sold are transformed into categorical features.
df_full['YrSold'] = df_full['YrSold'].astype(str)
df_full['MoSold'] = df_full['MoSold'].astype(str)
df_full.YrSold = pd.to_numeric(df_full.YrSold)
df_full.GarageYrBlt = pd.to_numeric(df_full.GarageYrBlt)
df_full['LotFrontage'] = df_full.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
#mean = df_full.query('LotConfig == "CulDSac" and BldgType =="1Fam"').groupby(['LotConfig','BldgType'])['LotFrontage'].mean()
#df_full.LotFrontage = df_full.LotFrontage.fillna(mean.values[0])
# df_full.Street.head(2)

In [121]:
objects = [col for col in df_full.columns if df_full[col].dtype == "object"]
df_full.update(df_full[objects].fillna('None'))
numerics = [col for col in df_full.columns if df_full[col].dtype != object]
df_full.update(df_full[numerics].fillna(0))
f = df_full.isna().sum()
f[f>0]

Series([], dtype: int64)

Feature engineering 

In [122]:
#df_full['CondTot'] = (df_full['Condition1'] + df_full['Condition2']) / 2
df_full['OverTot'] = df_full['OverallQual'] + df_full['OverallCond']
df_full['TotalSF'] = df_full['TotalBsmtSF'] + df_full['1stFlrSF'] + df_full['2ndFlrSF']
df_full['haspool'] = df_full['PoolArea'].apply(lambda x: 1 if x > 0 else 0)

Добавить в мой файл с фунциями, полезно для работы с skew

In [123]:
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax,yeojohnson
numerics = [col for col in df_full.columns if df_full[col].dtype != object]
skew_f = df_full[numerics].apply(lambda x: skew(x)).sort_values(ascending=False)
highest_skew =  skew_f[skew_f > 0.5]
highest_skew

MiscVal          21.947195
PoolArea         16.898328
haspool          14.884318
LotArea          12.822431
LowQualFinSF     12.088761
3SsnPorch        11.376065
KitchenAbvGr      4.302254
BsmtFinSF2        4.146143
EnclosedPorch     4.003891
ScreenPorch       3.946694
BsmtHalfBath      3.931594
MasVnrArea        2.613592
OpenPorchSF       2.535114
WoodDeckSF        1.842433
TotalSF           1.511479
LotFrontage       1.505704
1stFlrSF          1.469604
BsmtFinSF1        1.425230
GrLivArea         1.269358
TotalBsmtSF       1.156894
BsmtUnfSF         0.919339
2ndFlrSF          0.861675
TotRmsAbvGrd      0.758367
Fireplaces        0.733495
HalfBath          0.694566
BsmtFullBath      0.624832
OverallCond       0.570312
dtype: float64

In [124]:
for ind in highest_skew.index: 
    xt, lmbda = yeojohnson(df_full[ind])
    df_full[ind] = xt

In [125]:
train = df_full.iloc[:1459]
test = df_full.iloc[1460:]
train['Sqr'] = train['SalePrice'] / train['LotArea']
train['Sqr'] = train.groupby('Neighborhood')['Sqr'].transform(lambda x: x.median())
d = {}
for indice_fila, x_train in train.iterrows():
    d.update({x_train['Neighborhood']:x_train['Sqr']})
test['Sqr'] = 0.00
for indice, x_test in test.iterrows():
    test.loc[test.index == indice ,'Sqr'] = d[x_test['Neighborhood']]
df_full['Sqr'] = pd.concat([train.Sqr,test.Sqr])

In [126]:
df_full['MSSubClass'] = df_full['MSSubClass'].apply(str)
df_full['MoSold'] = df_full['MoSold'].astype(str) 
final_df_full = pd.get_dummies(df_full).reset_index(drop=True)

In [127]:
final_df_full.to_csv('prepared.csv')

In [128]:
# n_folds = 5
# def rmsle_cv(model):
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
#     rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
#     return(rmse)
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import cross_val_score
# clf = XGBRegressor(random_state=241)
# df = pd.get_dummies(df_full)
# X_test = df[df.LotFrontage.isna()].drop(columns = ['LotFrontage','SalePrice'])
# X_train  = df[df.LotFrontage.notna()].drop(columns = ['LotFrontage','SalePrice'])
# y_train = df['LotFrontage'][df.LotFrontage.notna()]
# Id = df['Id'][df.LotFrontage.isna()]
# clf.fit(X_train,y_train)
# predictions = clf.predict(X_test)

In [129]:
# df = pd.DataFrame(Id, columns = ['Id', 'LotFrontage'])
# df.LotFrontage = predictions
# df = df.set_index('Id')
# df_full.LotFrontage.fillna(df.LotFrontage,inplace=True)